<a href="https://colab.research.google.com/github/DongGwan0505/MNIST-classification-project/blob/main/MNIST_dataset_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# 랜덤 시드 고정
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# ===== 데이터 불러오기 & 전처리 =====
mnist = tf.keras.datasets.mnist
(x_train_full, y_train_full), (x_test, y_test) = mnist.load_data()

x_valid, x_train = x_train_full[:10000] / 255.0, x_train_full[10000:] / 255.0
y_valid, y_train = y_train_full[:10000], y_train_full[10000:]
x_test = x_test / 255.0

In [ ]:
# ===== 데이터 증강 설정 =====
datagen = ImageDataGenerator(
    rotation_range=10,      # 회전
    zoom_range=0.1,         # 확대/축소
    width_shift_range=0.1,  # 가로 이동
    height_shift_range=0.1  # 세로 이동
)
datagen.fit(x_train)

In [ ]:
# ===== 모델 정의 =====
def create_model(hidden_nodes=1024):
    model = Sequential([
        Flatten(input_shape=[28, 28]),
        Dense(hidden_nodes, activation="relu", kernel_initializer="he_normal"),
        BatchNormalization(),
        Dropout(0.3),

        Dense(hidden_nodes // 2, activation="relu", kernel_initializer="he_normal"),
        BatchNormalization(),
        Dropout(0.3),

        Dense(10, activation="softmax")
    ])
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="adam",  # Adam optimizer
        metrics=["accuracy"]
    )
    return model

In [ ]:
# ===== 모델 생성 =====
model = create_model(1024)

# ===== 콜백 설정 =====
checkpoint_cb = ModelCheckpoint("best_model.h5", save_best_only=True)
early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-4)

In [ ]:
# ===== 모델 학습 =====
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=100,
    validation_data=(x_valid, y_valid),
    callbacks=[checkpoint_cb, early_stopping_cb, reduce_lr_cb]
)

In [ ]:
# ===== 최종 평가 =====
model.evaluate(x_test, y_test)